In [19]:
import numpy as np
import matplotlib.pyplot as plt
from ax.service.ax_client import AxClient
from ax.service.utils.objective_properties import ObjectiveProperties
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Generators

# ✅ ZDT1多目的関数
def zdt1(params):
    x = np.array([params[f"x{i}"] for i in range(1, 31)])
    x1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / 29
    f1 = x1
    f2 = g * (1 - np.sqrt(x1 / g))
    return {"f1": f1, "f2": f2}

# ✅ パラメータ定義（x1〜x30 ∈ [0, 1]）
parameters = [{"name": f"x{i}", "type": "range", "bounds": [0.0, 1.0]} for i in range(1, 31)]

# ✅ 多目的最適化のGenerationStrategy
gs = GenerationStrategy(
    steps=[
        GenerationStep(model=Generators.SOBOL, num_trials=10),     # 初期点：準ランダム
        GenerationStep(model=Generators.MOO, num_trials=-1),       # 多目的BO（qEHVIなど）
    ]
)

# ✅ Axクライアント生成と実験初期化
client = AxClient(generation_strategy=gs)
client.create_experiment(
    name="zdt1_mobo",
    parameters=parameters,
    objectives={
        "f1": ObjectiveProperties(minimize=True),
        "f2": ObjectiveProperties(minimize=True),
    },
    is_test=True,
)

# ✅ 最適化ループ（評価・登録）
for cycle in range(5):
    print(f"\n📘 Cycle {cycle+1}")
    for _ in range(5):
        params, trial_index = client.get_next_trial()
        result = zdt1(params)
        client.complete_trial(
            trial_index=trial_index,
            raw_data={
                "f1": (result["f1"], 0.0),
                "f2": (result["f2"], 0.0),
            }
        )

    # 可視化
    pareto = client.get_pareto_optimal_parameters()
    pareto_f1 = [v[0]["f1"] for v in pareto.values()]
    pareto_f2 = [v[0]["f2"] for v in pareto.values()]

    data = client.experiment.fetch_data().df
    f1_vals = data[data["metric_name"] == "f1"]["mean"]
    f2_vals = data[data["metric_name"] == "f2"]["mean"]

    plt.figure(figsize=(6, 5))
    plt.scatter(f1_vals, f2_vals, c="blue", label="All points", alpha=0.5)
    plt.scatter(pareto_f1, pareto_f2, c="red", label="Pareto Front", s=80)
    plt.xlabel("f1")
    plt.ylabel("f2")
    plt.title(f"ZDT1 - Pareto Front (Cycle {cycle+1})")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


ModuleNotFoundError: No module named 'ax.service.utils.objective_properties'